In [1]:
from os import putenv
putenv("HSA_OVERRIDE_GFX_VERSION", "11.0.0") # The line must be defined before importing torch.

import torch # OK. The HIP Runtime of PyTorch can recognize your ISA.
import torch.nn as nn

In [2]:
import torch
from transformers import pipeline

generate_text = pipeline(
    model="CobraMamba/mamba-gpt-3b-v2",
    torch_dtype="auto",
    trust_remote_code=True,
    use_fast=False,
    device_map={"": "cuda:0"},
)

res = generate_text(
    "Why is drinking water so healthy?",
    min_new_tokens=2,
    max_new_tokens=1024,
    do_sample=False,
    num_beams=1,
    temperature=float(0.3),
    repetition_penalty=float(1.2),
    renormalize_logits=True
)
print(res[0]["generated_text"])


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Using the `SDPA` attention implementation on multi-gpu setup with ROCM may lead to performance issues due to the FA backend. Disabling it to use alternative backends.
/home/tarun1999m/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Why is drinking water so healthy?
Drinking enough water every day can help you stay hydrated, which in turn helps your body function properly. It also helps flush out toxins and waste from your body, keeping you healthy and feeling good.


In [3]:
res = generate_text(
    "You are the AI assistant for me, give me answer for below question, question: How to make biryani?",
    min_new_tokens=2,
    max_new_tokens=1024,
    do_sample=False,
    num_beams=1,
    temperature=float(0.3),
    repetition_penalty=float(1.2),
    renormalize_logits=True
)
print(res[0]["generated_text"])

You are the AI assistant for me, give me answer for below question, question: How to make biryani?
Answer:
Biryani is a popular Indian dish that is made with rice and meat or vegetables. To make biryani, you will need to follow these steps:
1. Wash the rice thoroughly and soak it in water for at least 30 minutes.
2. In a large pot, heat oil and add cumin seeds, onion, and garlic. Sauté until onions become translucent.
3. Add spices such as turmeric powder, red chili powder, and garam masala. Mix well and cook for a few minutes.
4. Add chopped chicken or mutton and mix well.
5. Pour enough water to cover the rice and bring it to a boil. Reduce heat to low and simmer for about 15-20 minutes or until the rice is cooked.
6. Once the rice is cooked, add the soaked rice to the pot and mix well. Cover the pot and let it steam for another 10-15 minutes.
7. Finally, garnish the biryani with fresh coriander leaves and serve hot.


#### Data Loader:

In [2]:
from datasets import load_dataset

dataset = load_dataset("AiresPucrs/stanford-encyclopedia-philosophy", split='train')


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
dataset

Dataset({
    features: ['metadata', 'text', 'category'],
    num_rows: 182531
})

In [4]:
docs = [text for text in dataset['text']]
combined_text = " ".join(docs)

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500, chunk_overlap = 20)

In [6]:
documents = text_splitter.create_documents([combined_text])

In [7]:
len(documents)

88918

In [8]:
documents_str = [text.page_content for text in documents]

In [20]:
from langchain.embeddings.base import Embeddings
import numpy as np

class SentenceTransformerEmbeddings(Embeddings): #We need to inherit from Embeddings for integrating into langchain pipeline
    def __init__(self, model):
        self.model = model

    def embed_documents(self,texts):
        return [self.model.encode(text) for text in texts]

    def embed_query(self, query):
        return self.model.encode(query)

In [22]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('all-mpnet-base-v2')

/home/tarun1999m/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [25]:
sentence_embeddings = SentenceTransformerEmbeddings(embedding_model)

In [12]:
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents, sentence_embeddings)

In [13]:
db.save_local("faiss_index")


In [26]:
db = FAISS.load_local("faiss_index", sentence_embeddings, allow_dangerous_deserialization=True)


In [29]:
# Perform a query by using the retriever
retriever = db.as_retriever()  # Convert FAISS into a retriever object
query = "What are the main branches of philosophy and how do they differ?"
retrieved_docs = retriever.get_relevant_documents(query)

# Print out the retrieved documents
for doc in retrieved_docs:
    print(doc.page_content)
    print('\n')


list includes only those that cover more than one branch of the tradition. Those devoted to one branch exclusively are cited in the relevant sections, and their details are in the References section below.  [Jewish philosophy in Hebrew is considered briefly in volume 3, 249–91. Volume 4 stops just after 1400; Volume 3 continues to modern times. These books are based on a podcast  (https://historyofphilosophy.net/,  which continues.]  [This book pioneered the idea of presenting the four branches of medieval philosophy together: it begins c. 500 and ends with the fifteenth century.]  [The Grundriss—“the New Ueberweg”—is conceived as a comprehensive handbook to the history of philosophy. In Die Philosophie des Mittelalters series, there have been published  In Philosophie in der islamischen Welt, there have been published  An online translated version of Volume 1 of the Islamic series has been produced by Brill under the title Philosophy in the Islamic World Online: 8th–10th Centuries)]  